# Notebook for Annotation of clusters at `Condition` level after batch correction using `bbknn`

**Created by :** Srivalli Kolla

**Created on :** 05 May, 2025

**Modified on :** 05 May, 2025

**University of Würzburg**

Env : scanpy (Python 3.12.2)

# Importing Packages

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sb
import datetime
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import rcParams

In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()

plt.rcParams['figure.dpi'] = 300  
plt.rcParams['savefig.dpi'] = 300

timestamp = datetime.datetime.now().strftime("%d_%m_%y")

# Data import

In [3]:
adata = sc.read_h5ad('./Github/ACM_sn_2025/data/acm_bbknn_batch_corrected_01_05_25.h5ad')
adata

#### Check if data is raw or Normalized

In [4]:
def X_is_raw(adata):
    return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

In [5]:
print(X_is_raw(adata))

In [6]:
adata.X= adata.layers['cpm_normalization']
print(X_is_raw(adata))

# Data Seperation - Condition level

In [7]:
adata.obs['Condition'].value_counts()

In [8]:
Ctr_noninf = adata[adata.obs['Condition'] == 'Ctr_noninf'].copy()
Ctr_noninf

# Annotation

## Clustering

In [9]:
sc.tl.pca(Ctr_noninf,  n_comps=50, use_highly_variable=True)
sc.pp.neighbors(Ctr_noninf)

In [10]:
sc.tl.umap(Ctr_noninf)

## Data Visualization

In [11]:
sc.pl.umap(Ctr_noninf,color= ['Sample_Name', 'Sex', 'Genotype', 'Treatment', 'Condition', 'Sample_ID', 'n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo', 'doublet_scores', 'XIST-percentage', 'gender_check_cov', 'phase'], frameon = False,layer = 'cpm_normalization', cmap = 'RdYlBu_r' )

## Marker Genes

In [12]:
marker_genes = {'Ventricular Cardiomyocytes' : ['Myh7',' Myl2',' Fhl2'],
 'Atrial Cardiomyocytes' :['Nppa',' Myl7',' Myl4'],
 'Fibroblasts': ['Dcn',' Gsn',' Pdgfra'],
 'Endothelial Cells' :['Vwf',' Pecam1',' Cdh5'],
 'Pericytes' :['Rgs5',' Abcc9',' Kcnj8'],
 'Smooth Muscle Cells' :['Myh11',' Tagln',' Acta2'],
 'Myeloid Immune Cells' : ['Cd14',' C1qa',' Cd68'],
 'Lymphoid Immune Cells' :['Cd8a',' Il7r',' Cd40lg'],
 'Adipocytes' : ['Gpam',' Fasn',' Lep'],
 'Neuronal Cells' :['Plp1',' Nrxn1',' Nrxn3'],
 'Mesothelial Cells' :['Msln',' Wt1',' Bnc1']
}

In [13]:
marker_genes_in_data = {}
for ct, markers in marker_genes.items():
    markers_found = []
    for marker in markers:
        if marker in Ctr_noninf.var.index:
            markers_found.append(marker)
    marker_genes_in_data[ct] = markers_found

## Plotting

In [14]:
for cell_type, genes in marker_genes.items():
    
    cleaned_genes = [g.strip() for g in genes if g.strip() in Ctr_noninf.var_names]

    if cleaned_genes:
        print(f"{cell_type.upper()}:\n  → Plotting: {', '.join(cleaned_genes)}\n")
        sc.pl.umap(
            Ctr_noninf,
            color=cleaned_genes,
            vmin=0,
            vmax="p99",
            sort_order=False,
            frameon=False,
            cmap="RdYlBu_r", layer = 'cpm_normalization'
        )
    else:
        print(f"{cell_type.upper()}:\n  ✗ No valid marker genes found in Ctr_noninf.var_names.\n")

    print("\n" + "-"*60 + "\n")

## Leiden Clustering

In [15]:
sc.tl.leiden(Ctr_noninf, resolution=1, key_added="leiden_1")

In [16]:
sc.tl.leiden(Ctr_noninf, resolution=0.1, key_added="leiden_0.1")

In [17]:
sc.tl.leiden(Ctr_noninf, resolution=0.2, key_added="leiden_0.2")

In [18]:
sc.tl.leiden(Ctr_noninf, resolution=0.3, key_added="leiden_0.3")

In [19]:
sc.tl.leiden(Ctr_noninf, resolution=0.5, key_added="leiden_0.5")

In [20]:
sc.pl.umap(Ctr_noninf, color=["leiden_0.1","leiden_0.2","leiden_0.3","leiden_0.5","leiden_1"],frameon= False,legend_loc="on data")

# Cluster Annotation

In [21]:
cl_annotation = {
"0" : "Endothelial Cells",
"1" : "Venticular Cardiomyocytes",
"2" : "Venticular Cardiomyocytes",
"3" : "Fibroblasts",
"4" : "Myeloid Immune Cells",
"5" : "Endothelial Cells + + Neuronal Cells",
"6" : "Fibroblasts + Endothelial Cells",
"7" : "Pericytes + Smooth Muscle Cells" ,
"8" : "Endothelial Cells",
"9" : "Endothelial Cells + Neuronal Cells",
"10" : "Lymphoid Immune Cells",
"11" : "Venticular Cardiomyocytes + Atrial Cardiomyocytes + Fibroblasts ",
"12" : "Venticular Cardiomyocytes + Atrial Cardiomyocytes + Fibroblasts"
}

In [22]:
Ctr_noninf.obs["manual_celltype_annotation_specific"] = Ctr_noninf.obs['leiden_0.3'].map(cl_annotation)

In [23]:
sc.pl.umap(Ctr_noninf, color = 'leiden_0.3',frameon= False, legend_loc = 'on data')

In [24]:
sc.pl.umap(Ctr_noninf, color = ["manual_celltype_annotation_specific"], frameon = False)

In [25]:
cl_annotation2 = {
"0" : "Endothelial Cells",
"1" : "Venticular Cardiomyocytes",
"2" : "Venticular Cardiomyocytes",
"3" : "Fibroblasts",
"4" : "Myeloid Immune Cells",
"5" : "Mixed Cell Types",
"6" : "Mixed Cell Types",
"7" : "Pericytes + Smooth Muscle Cells" ,
"8" : "Endothelial Cells",
"9" : "Mixed Cell Types",
"10" : "Lymphoid Immune Cells",
"11" : "Mixed Cell Types",
"12" : "Mixed Cell Types"
}

In [26]:
Ctr_noninf.obs["manual_celltype_annotation_broad"] = Ctr_noninf.obs['leiden_0.3'].map(cl_annotation2)

In [27]:
sc.pl.umap(Ctr_noninf, color = ["manual_celltype_annotation_broad"], frameon = False)

# Differentially Expressed Genes (DEGs)

In [28]:
sc.tl.rank_genes_groups(Ctr_noninf, groupby="leiden_0.3", method="wilcoxon",use_raw= False, key_added = 'dea_leiden')

In [29]:
sc.tl.dendrogram(Ctr_noninf,groupby='leiden_0.3')
sc.pl.rank_genes_groups_dotplot(Ctr_noninf,groupby='leiden_0.3',standard_scale ='var',n_genes= 5, key='dea_leiden',values_to_plot= 'scores')

In [30]:
deg_df = pd.DataFrame(Ctr_noninf.uns["dea_leiden"]["names"]).head(100)  
print(deg_df)

In [31]:
deg_df.to_csv(f'./Github/ACM_sn_2025/data/DE_genes_bbknn_toppfun_Ctr_noninf_{timestamp}.csv',sep=',')

# Cluster Annotation - based on DEGs

In [32]:
toppfun_annotation = {
"0":"Endothelial",
"1":"Cytoplasmic Cardiomyocyte",
"2":"Ventricular Cardiomyocyte",
"3":"Fibroblasts",
"4":"Macrophages",
"5":"Endothelial",
"6":"Macrophages",
"7":"Mesenchymal cells",
"8":"Endothelial",
"9":"Neuronal cells",
"10":"Unclear",
"11":"Unclear",
"12":"Unclear",
}

In [33]:
Ctr_noninf.obs["toppfun_annotation"] = Ctr_noninf.obs['leiden_0.3'].map(toppfun_annotation)

In [34]:
sc.pl.umap(Ctr_noninf, color = ["toppfun_annotation"], frameon = False)

In [35]:
Ctr_noninf.write_h5ad(f'./Github/ACM_sn_2025/data/acm_manual_anno_Ctr_noninf_{timestamp}.h5ad')